## Larger objects within clusters

In [ ]:
import argparse
import pickle
import os
import sys
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn.covariance import EllipticEnvelope
import matplotlib.pyplot as plt
from scipy.spatial.distance import mahalanobis

scene_synthesis_path = 'path_to_scene_synthesis'

sys.path.append(os.path.dirname(scene_synthesis_path))
sys.path.append(scene_synthesis_path)

from scene_synthesis.datasets.threed_front import ThreedFront
from scene_synthesis.datasets.threed_future_dataset import ThreedFutureDataset
from scene_synthesis.utils import get_textured_objects, get_floor_plan
from utils import export_scene, floor_plan_from_scene, render_scene_from_bbox_params, render_to_folder, render

class_labels_dining = [
    "armchair", "bookshelf", "cabinet", "ceiling_lamp", 
    "chaise_longue_sofa", "chinese_chair", "coffee_table", 
    "console_table", "corner_side_table", "desk", "dining_chair", 
    "dining_table", "l_shaped_sofa", "lazy_sofa", "lounge_chair", 
    "loveseat_sofa", "multi_seat_sofa", "pendant_lamp", 
    "round_end_table", "shelf", "stool", "tv_stand", 
    "wardrobe", "wine_cabinet", "start", "end"
]

class_labels_bedroom = ["armchair", "bookshelf", "cabinet", "ceiling_lamp", "chair", 
                        "children_cabinet", "coffee_table", "desk", "double_bed", 
                        "dressing_chair", "dressing_table", "kids_bed", "nightstand", 
                        "pendant_lamp", "shelf", "single_bed", "sofa", "stool", "table", 
                        "tv_stand", "wardrobe", "start", "end"]

def fetch_scene_id(all_scene_paths, scene_id):
    for scene in all_scene_paths:
        if scene_id in scene:
            return scene
    return None

def load_attributes_from_npz(scene_path, attrib_list=[]):
    scene = np.load(os.path.join(scene_path, 'boxes.npz'))
    if not attrib_list:
        attrib_list = list(scene.keys())
    vals = {}
    for attrib in attrib_list:
        vals[attrib] = scene[attrib]
    return vals

def calculate_giou(bbox1, bbox2):
    def area(bbox):
        return max(0, bbox[2] - bbox[0]) * max(0, bbox[3] - bbox[1])

    xA = max(bbox1[0], bbox2[0])
    yA = max(bbox1[1], bbox2[1])
    xB = min(bbox1[2], bbox2[2])
    yB = min(bbox1[3], bbox2[3])
    intersection = area([xA, yA, xB, yB])
    
    union = area(bbox1) + area(bbox2) - intersection
    
    iou = intersection / union if union > 0 else 0
    
    xC = min(bbox1[0], bbox2[0])
    yC = min(bbox1[1], bbox2[1])
    xD = max(bbox1[2], bbox2[2])
    yD = max(bbox1[3], bbox2[3])
    enclosing_area = area([xC, yC, xD, yD])
    
    giou = iou - ((enclosing_area - union) / enclosing_area)
    return giou

def dist_matrix(translations, sizes, lambda_value=0.02):
    num_boxes = len(translations)
    distance_matrix = np.zeros((num_boxes, num_boxes))
    
    for i in range(num_boxes):
        for j in range(num_boxes):
            if i != j:
                bbox_i = [
                    translations[i][0] - sizes[i][0] / 2,
                    translations[i][1] - sizes[i][1] / 2,
                    translations[i][0] + sizes[i][0] / 2,
                    translations[i][1] + sizes[i][1] / 2
                ]
                
                bbox_j = [
                    translations[j][0] - sizes[j][0] / 2,
                    translations[j][1] - sizes[j][1] / 2,
                    translations[j][0] + sizes[j][0] / 2,
                    translations[j][1] + sizes[j][1] / 2
                ]
                
                center_i = (translations[i][0], translations[i][1])
                center_j = (translations[j][0], translations[j][1])
                euclidean_distance = np.linalg.norm(np.array(center_i) - np.array(center_j))
                
                giou = calculate_giou(bbox_i, bbox_j)
                
                distance_matrix[i, j] = euclidean_distance + lambda_value * (1 - giou)

    return distance_matrix

def extract_features_from_bboxes(attrib):
    class_labels = attrib["class_labels"].argmax(-1)
    translations = attrib["translations"]
    sizes = attrib["sizes"]
    angles = np.squeeze(attrib["angles"])
    
    features_translations = translations[:,:2]
    features_sizes = sizes[:,:2]
    return features_translations, features_sizes

def cluster_scene_objects_gmm(distance_matrix, n_components=5):
    gmm = GaussianMixture(n_components=n_components, covariance_type='full')
    cluster_labels = gmm.fit_predict(distance_matrix)
    return cluster_labels

def detect_outliers_elliptic_envelope(features, contamination=0.1):
    scaler = StandardScaler()
    features_std = scaler.fit_transform(features)

    ee = EllipticEnvelope(contamination=contamination)
    ee.fit(features_std)
    
    outlier_pred = ee.predict(features_std)
    
    outliers = np.where(outlier_pred == -1)[0]
    
    decision_scores = ee.decision_function(features_std)
    
    return outliers, decision_scores

def find_largest_object_in_clusters(features_sizes, cluster_labels, class_labels):
    cluster_largest_objects = {}
    print(features_sizes)
    areas = np.prod(features_sizes, axis=1)
    print(areas)
    unique_clusters = np.unique(cluster_labels)
    for cluster in unique_clusters:
        cluster_indices = np.where(cluster_labels == cluster)[0]
        cluster_areas = areas[cluster_indices]
        
        largest_index_within_cluster = cluster_indices[np.argmax(cluster_areas)]
        largest_class_label = class_labels[largest_index_within_cluster]
        cluster_largest_objects[cluster] = {
            "index": largest_index_within_cluster,
            "area": cluster_areas.max(),
            "label": largest_class_label
        }
    
    return cluster_largest_objects

def main(scene_id):
    processed_path = '/home/gauravr/Desktop/IFA/code/preprocessed/3d_front_npz/'
    if not os.path.exists(processed_path):
        print(f"Processed path {processed_path} does not exist.")
        sys.exit(1)
    
    all_paths = [os.path.join(processed_path, dir_) for dir_ in os.listdir(processed_path)]
    
    all_scenes = []
    for path_ in all_paths:
        if os.path.isdir(path_):
            all_scenes.extend([os.path.join(path_, x) for x in os.listdir(path_) if os.path.isdir(os.path.join(path_, x))])
    
    scene_path = fetch_scene_id(all_scenes, scene_id)
    if scene_path is None:
        print(f"Scene ID {scene_id} not found.")
        sys.exit(1)

    required_attribs = ['class_labels', 'translations', 'sizes', 'angles']
    attribs = load_attributes_from_npz(scene_path, required_attribs)
    
    for attrib in required_attribs:
        if attrib not in attribs:
            print(f"Attribute '{attrib}' not found in the NPZ file.")
            sys.exit(1)
    
    features_translations, features_sizes = extract_features_from_bboxes(attribs)

    total_objects = attribs['class_labels'].shape[0]
    print(f"Total number of objects in the scene: {total_objects}")

    distance_matrix = dist_matrix(features_translations, features_sizes)

    n_components = min(2, total_objects)
    cluster_labels = cluster_scene_objects_gmm(distance_matrix, n_components=n_components)

    class_labels = attribs['class_labels'].argmax(-1)

    print("\nClass labels for all objects in the scene:")
    for i, label_idx in enumerate(class_labels):
        print(f"Object {i}: {class_labels_dining[label_idx]} (Class Index: {label_idx})")

    print(f"\nClustering results using GMM:")
    cluster_dict = {}

    for idx, label in enumerate(cluster_labels):
        if label not in cluster_dict:
            cluster_dict[label] = []
        cluster_dict[label].append(idx)

    # Print objects in each cluster with labels
    for label, indices in cluster_dict.items():
        print(f"\nCluster {label} contains the following objects:")
        for idx in indices:
            print(f"  Object {idx}: {class_labels_dining[class_labels[idx]]}")

    print("\nDetecting outliers using Elliptic Envelope...")
    features = np.hstack((features_translations, features_sizes))
    outliers, decision_scores = detect_outliers_elliptic_envelope(features)
    
    print(f"\nOutliers detected at indices: {outliers}")
    print(f"Elliptic Envelope decision scores: {decision_scores}")

    cluster_largest_objects = find_largest_object_in_clusters(features_sizes, cluster_labels, class_labels)

    print("\nLargest objects in each cluster by size:")
    for cluster, info in cluster_largest_objects.items():
        label_name = class_labels_dining[info['label']]
        print(f"Cluster {cluster}: Largest Object Index {info['index']} with Area {info['area']} and Label '{label_name}'")

if __name__ == "__main__":    
    main(scene_id='LivingDiningRoom-941')


Total number of objects in the scene: 13

Class labels for all objects in the scene:
Object 0: multi_seat_sofa (Class Index: 16)
Object 1: coffee_table (Class Index: 6)
Object 2: armchair (Class Index: 0)
Object 3: corner_side_table (Class Index: 8)
Object 4: pendant_lamp (Class Index: 17)
Object 5: dining_chair (Class Index: 10)
Object 6: dining_chair (Class Index: 10)
Object 7: dining_chair (Class Index: 10)
Object 8: dining_table (Class Index: 11)
Object 9: pendant_lamp (Class Index: 17)
Object 10: console_table (Class Index: 7)
Object 11: dining_chair (Class Index: 10)
Object 12: stool (Class Index: 20)

Clustering results using GMM:

Cluster 1 contains the following objects:
  Object 0: multi_seat_sofa
  Object 1: coffee_table
  Object 2: armchair
  Object 4: pendant_lamp
  Object 12: stool

Cluster 0 contains the following objects:
  Object 3: corner_side_table
  Object 5: dining_chair
  Object 6: dining_chair
  Object 7: dining_chair
  Object 8: dining_table
  Object 9: pendant_

/home/gauravr/anaconda3/envs/physgen/lib/python3.9/site-packages/sklearn/covariance/_robust_covariance.py:187: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-5.870283065003756 > -40.280113148001455). You may want to try with a higher value of support_fraction (current value: 0.692).
  warnings.warn(
/home/gauravr/anaconda3/envs/physgen/lib/python3.9/site-packages/sklearn/covariance/_robust_covariance.py:187: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-5.982659322280981 > -42.617844626450328). You may want to try with a higher value of support_fraction (current value: 0.692).
  warnings.warn(
/home/gauravr/anaconda3/envs/physgen/lib/python3.9/site-packages/sklearn/covariance/_robust_covariance.py:187: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-5.870283065003756 > -40.280113148001455). You may want to try with a higher value of su

## Estimate tree within clusters

In [ ]:
import argparse
import pickle
import os
import sys
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn.covariance import EllipticEnvelope
import matplotlib.pyplot as plt
from scipy.spatial.distance import mahalanobis

scene_synthesis_path = 'path_to_scene_synthesis'

sys.path.append(os.path.dirname(scene_synthesis_path))
sys.path.append(scene_synthesis_path)

from scene_synthesis.datasets.threed_front import ThreedFront
from scene_synthesis.datasets.threed_future_dataset import ThreedFutureDataset
from scene_synthesis.utils import get_textured_objects, get_floor_plan
from utils import export_scene, floor_plan_from_scene, render_scene_from_bbox_params, render_to_folder, render

class_labels_dining = [
    "armchair", "bookshelf", "cabinet", "ceiling_lamp", 
    "chaise_longue_sofa", "chinese_chair", "coffee_table", 
    "console_table", "corner_side_table", "desk", "dining_chair", 
    "dining_table", "l_shaped_sofa", "lazy_sofa", "lounge_chair", 
    "loveseat_sofa", "multi_seat_sofa", "pendant_lamp", 
    "round_end_table", "shelf", "stool", "tv_stand", 
    "wardrobe", "wine_cabinet", "start", "end"
]

class_labels_bedroom = ["armchair", "bookshelf", "cabinet", "ceiling_lamp", "chair", 
                        "children_cabinet", "coffee_table", "desk", "double_bed", 
                        "dressing_chair", "dressing_table", "kids_bed", "nightstand", 
                        "pendant_lamp", "shelf", "single_bed", "sofa", "stool", "table", 
                        "tv_stand", "wardrobe", "start", "end"]

def fetch_scene_id(all_scene_paths, scene_id):
    for scene in all_scene_paths:
        if scene_id in scene:
            return scene
    return None

def load_attributes_from_npz(scene_path, attrib_list=[]):
    scene = np.load(os.path.join(scene_path, 'boxes.npz'))
    if not attrib_list:
        attrib_list = list(scene.keys())
    vals = {}
    for attrib in attrib_list:
        vals[attrib] = scene[attrib]
    return vals

def calculate_giou(bbox1, bbox2):
    def area(bbox):
        return max(0, bbox[2] - bbox[0]) * max(0, bbox[3] - bbox[1])

    xA = max(bbox1[0], bbox2[0])
    yA = max(bbox1[1], bbox2[1])
    xB = min(bbox1[2], bbox2[2])
    yB = min(bbox1[3], bbox2[3])
    intersection = area([xA, yA, xB, yB])
    
    union = area(bbox1) + area(bbox2) - intersection
    
    iou = intersection / union if union > 0 else 0
    
    xC = min(bbox1[0], bbox2[0])
    yC = min(bbox1[1], bbox2[1])
    xD = max(bbox1[2], bbox2[2])
    yD = max(bbox1[3], bbox2[3])
    enclosing_area = area([xC, yC, xD, yD])
    
    giou = iou - ((enclosing_area - union) / enclosing_area)
    return giou

def dist_matrix(translations, sizes, lambda_value=0.02):
    num_boxes = len(translations)
    distance_matrix = np.zeros((num_boxes, num_boxes))
    
    for i in range(num_boxes):
        for j in range(num_boxes):
            if i != j:
                bbox_i = [
                    translations[i][0] - sizes[i][0] / 2,
                    translations[i][1] - sizes[i][1] / 2,
                    translations[i][0] + sizes[i][0] / 2,
                    translations[i][1] + sizes[i][1] / 2
                ]
                
                bbox_j = [
                    translations[j][0] - sizes[j][0] / 2,
                    translations[j][1] - sizes[j][1] / 2,
                    translations[j][0] + sizes[j][0] / 2,
                    translations[j][1] + sizes[j][1] / 2
                ]
                
                center_i = (translations[i][0], translations[i][1])
                center_j = (translations[j][0], translations[j][1])
                euclidean_distance = np.linalg.norm(np.array(center_i) - np.array(center_j))
                
                giou = calculate_giou(bbox_i, bbox_j)
                
                distance_matrix[i, j] = euclidean_distance + lambda_value * (1 - giou)

    return distance_matrix

def extract_features_from_bboxes(attrib):
    class_labels = attrib["class_labels"].argmax(-1)
    translations = attrib["translations"]
    sizes = attrib["sizes"]
    angles = np.squeeze(attrib["angles"])
    
    features_translations = translations[:,:2]
    features_sizes = sizes[:,:2]
    return features_translations, features_sizes

def cluster_scene_objects_gmm(distance_matrix, n_components=5):
    gmm = GaussianMixture(n_components=n_components, covariance_type='full')
    cluster_labels = gmm.fit_predict(distance_matrix)
    return cluster_labels

def detect_outliers_elliptic_envelope(features, contamination=0.1):
    scaler = StandardScaler()
    features_std = scaler.fit_transform(features)

    ee = EllipticEnvelope(contamination=contamination)
    ee.fit(features_std)
    
    outlier_pred = ee.predict(features_std)
    
    outliers = np.where(outlier_pred == -1)[0]
    
    decision_scores = ee.decision_function(features_std)
    
    return outliers, decision_scores

def find_largest_object_in_clusters(features_sizes, cluster_labels, class_labels):
    cluster_largest_objects = {}
    areas = np.prod(features_sizes, axis=1)
    unique_clusters = np.unique(cluster_labels)
    for cluster in unique_clusters:
        cluster_indices = np.where(cluster_labels == cluster)[0]
        cluster_areas = areas[cluster_indices]
        
        largest_index_within_cluster = cluster_indices[np.argmax(cluster_areas)]
        largest_class_label = class_labels[largest_index_within_cluster]
        cluster_largest_objects[cluster] = {
            "index": largest_index_within_cluster,
            "area": cluster_areas.max(),
            "label": largest_class_label
        }
    
    return cluster_largest_objects

def build_tree(cluster_largest_objects, cluster_labels, features_sizes, class_labels, label_names):
    cluster_trees = {}
    for cluster, info in cluster_largest_objects.items():
        root_label = label_names[info['label']]
        root_index = info['index']
        
        # Initialize the root node of the tree with the largest object
        tree = {root_index: {"label": root_label, "children": []}}
        cluster_indices = np.where(cluster_labels == cluster)[0]
        
        for idx in cluster_indices:
            if idx == root_index:
                continue  # skip the root node
            
            child_label = label_names[class_labels[idx]]
            
            # Attach all items directly under the root node
            tree[root_index]["children"].append({"index": idx, "label": child_label, "children": []})
        
        cluster_trees[cluster] = tree

    return cluster_trees

def main(scene_id):
    processed_path = '/home/gauravr/Desktop/IFA/code/preprocessed/3d_front_npz/'
    if not os.path.exists(processed_path):
        print(f"Processed path {processed_path} does not exist.")
        sys.exit(1)
    
    all_paths = [os.path.join(processed_path, dir_) for dir_ in os.listdir(processed_path)]
    
    all_scenes = []
    for path_ in all_paths:
        if os.path.isdir(path_):
            all_scenes.extend([os.path.join(path_, x) for x in os.listdir(path_) if os.path.isdir(os.path.join(path_, x))])
    
    scene_path = fetch_scene_id(all_scenes, scene_id)
    if scene_path is None:
        print(f"Scene ID {scene_id} not found.")
        sys.exit(1)

    required_attribs = ['class_labels', 'translations', 'sizes', 'angles']
    attribs = load_attributes_from_npz(scene_path, required_attribs)
    
    for attrib in required_attribs:
        if attrib not in attribs:
            print(f"Attribute '{attrib}' not found in the NPZ file.")
            sys.exit(1)
    
    features_translations, features_sizes = extract_features_from_bboxes(attribs)

    total_objects = attribs['class_labels'].shape[0]
    print(f"Total number of objects in the scene: {total_objects}")

    distance_matrix = dist_matrix(features_translations, features_sizes)

    n_components = min(2, total_objects)
    cluster_labels = cluster_scene_objects_gmm(distance_matrix, n_components=n_components)

    class_labels = attribs['class_labels'].argmax(-1)

    print("\nClass labels for all objects in the scene:")
    for i, label_idx in enumerate(class_labels):
        print(f"Object {i}: {class_labels_dining[label_idx]} (Class Index: {label_idx})")

    print(f"\nClustering results using GMM:")
    cluster_dict = {}

    for idx, label in enumerate(cluster_labels):
        if label not in cluster_dict:
            cluster_dict[label] = []
        cluster_dict[label].append(idx)

    for label, indices in cluster_dict.items():
        print(f"\nCluster {label} contains the following objects:")
        for idx in indices:
            print(f"  Object {idx}: {class_labels_dining[class_labels[idx]]}")

    print("\nDetecting outliers using Elliptic Envelope...")
    features = np.hstack((features_translations, features_sizes))
    outliers, decision_scores = detect_outliers_elliptic_envelope(features)
    
    print(f"\nOutliers detected at indices: {outliers}")
    print(f"Elliptic Envelope decision scores: {decision_scores}")

    cluster_largest_objects = find_largest_object_in_clusters(features_sizes, cluster_labels, class_labels)

    print("\nLargest objects in each cluster by size:")
    for cluster, info in cluster_largest_objects.items():
        label_name = class_labels_dining[info['label']]
        print(f"Cluster {cluster}: Largest Object Index {info['index']} with Area {info['area']} and Label '{label_name}'")

    cluster_trees = build_tree(cluster_largest_objects, cluster_labels, features_sizes, class_labels, class_labels_dining)

    for cluster, tree in cluster_trees.items():
        print(f"\nCluster {cluster} Tree Structure:")
        for node, data in tree.items():
            print(f"Root: Object {node} ({data['label']})")
            for child in data["children"]:
                print(f"  └─ Child Object {child['index']} ({child['label']})")

if __name__ == "__main__":    
    main(scene_id='LivingDiningRoom-8302')


Total number of objects in the scene: 16

Class labels for all objects in the scene:
Object 0: multi_seat_sofa (Class Index: 16)
Object 1: loveseat_sofa (Class Index: 15)
Object 2: pendant_lamp (Class Index: 17)
Object 3: tv_stand (Class Index: 21)
Object 4: bookshelf (Class Index: 1)
Object 5: coffee_table (Class Index: 6)
Object 6: dining_table (Class Index: 11)
Object 7: dining_chair (Class Index: 10)
Object 8: dining_chair (Class Index: 10)
Object 9: dining_chair (Class Index: 10)
Object 10: dining_chair (Class Index: 10)
Object 11: dining_chair (Class Index: 10)
Object 12: dining_chair (Class Index: 10)
Object 13: shelf (Class Index: 19)
Object 14: bookshelf (Class Index: 1)
Object 15: pendant_lamp (Class Index: 17)

Clustering results using GMM:

Cluster 1 contains the following objects:
  Object 0: multi_seat_sofa
  Object 1: loveseat_sofa
  Object 2: pendant_lamp
  Object 3: tv_stand
  Object 5: coffee_table
  Object 14: bookshelf

Cluster 0 contains the following objects:
  Ob

In [ ]:
import argparse
import pickle
import os
import sys
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn.covariance import EllipticEnvelope
import matplotlib.pyplot as plt
from scipy.spatial.distance import mahalanobis

scene_synthesis_path = 'path_to_scene_synthesis'

sys.path.append(os.path.dirname(scene_synthesis_path))
sys.path.append(scene_synthesis_path)

from scene_synthesis.datasets.threed_front import ThreedFront
from scene_synthesis.datasets.threed_future_dataset import ThreedFutureDataset
from scene_synthesis.utils import get_textured_objects, get_floor_plan
from utils import export_scene, floor_plan_from_scene, render_scene_from_bbox_params, render_to_folder, render

class_labels_dining = [
    "armchair", "bookshelf", "cabinet", "ceiling_lamp", 
    "chaise_longue_sofa", "chinese_chair", "coffee_table", 
    "console_table", "corner_side_table", "desk", "dining_chair", 
    "dining_table", "l_shaped_sofa", "lazy_sofa", "lounge_chair", 
    "loveseat_sofa", "multi_seat_sofa", "pendant_lamp", 
    "round_end_table", "shelf", "stool", "tv_stand", 
    "wardrobe", "wine_cabinet", "start", "end"]

class_labels_bedroom = [
    "armchair", "bookshelf", "cabinet", "ceiling_lamp", "chair", 
    "children_cabinet", "coffee_table", "desk", "double_bed", 
    "dressing_chair", "dressing_table", "kids_bed", "nightstand", 
    "pendant_lamp", "shelf", "single_bed", "sofa", "stool", "table", 
    "tv_stand", "wardrobe", "start", "end"]

def fetch_scene_id(all_scene_paths, scene_id):
    for scene in all_scene_paths:
        if scene_id in scene:
            return scene
    return None

def load_attributes_from_npz(scene_path, attrib_list=[]):
    scene = np.load(os.path.join(scene_path, 'boxes.npz'))
    if not attrib_list:
        attrib_list = list(scene.keys())
    vals = {}
    for attrib in attrib_list:
        vals[attrib] = scene[attrib]
    return vals

def calculate_giou(bbox1, bbox2):
    def area(bbox):
        return max(0, bbox[2] - bbox[0]) * max(0, bbox[3] - bbox[1])

    xA = max(bbox1[0], bbox2[0])
    yA = max(bbox1[1], bbox2[1])
    xB = min(bbox1[2], bbox2[2])
    yB = min(bbox1[3], bbox2[3])
    intersection = area([xA, yA, xB, yB])
    
    union = area(bbox1) + area(bbox2) - intersection
    
    iou = intersection / union if union > 0 else 0
    
    xC = min(bbox1[0], bbox2[0])
    yC = min(bbox1[1], bbox2[1])
    xD = max(bbox1[2], bbox2[2])
    yD = max(bbox1[3], bbox2[3])
    enclosing_area = area([xC, yC, xD, yD])
    
    giou = iou - ((enclosing_area - union) / enclosing_area)
    return giou

def dist_matrix(translations, sizes, lambda_value=0.02):
    num_boxes = len(translations)
    distance_matrix = np.zeros((num_boxes, num_boxes))
    
    for i in range(num_boxes):
        for j in range(num_boxes):
            if i != j:
                bbox_i = [
                    translations[i][0] - sizes[i][0] / 2,
                    translations[i][1] - sizes[i][1] / 2,
                    translations[i][0] + sizes[i][0] / 2,
                    translations[i][1] + sizes[i][1] / 2
                ]
                
                bbox_j = [
                    translations[j][0] - sizes[j][0] / 2,
                    translations[j][1] - sizes[j][1] / 2,
                    translations[j][0] + sizes[j][0] / 2,
                    translations[j][1] + sizes[j][1] / 2
                ]
                
                center_i = (translations[i][0], translations[i][1])
                center_j = (translations[j][0], translations[j][1])
                euclidean_distance = np.linalg.norm(np.array(center_i) - np.array(center_j))
                
                giou = calculate_giou(bbox_i, bbox_j)
                
                distance_matrix[i, j] = euclidean_distance + lambda_value * (1 - giou)

    return distance_matrix

def extract_features_from_bboxes(attrib):
    class_labels = attrib["class_labels"].argmax(-1)
    translations = attrib["translations"]
    sizes = attrib["sizes"]
    angles = np.squeeze(attrib["angles"])
    
    features_translations = translations[:,:2]
    features_sizes = sizes[:,:2]
    return features_translations, features_sizes

def cluster_scene_objects_gmm(distance_matrix, n_components=5):
    gmm = GaussianMixture(n_components=n_components, covariance_type='full')
    cluster_labels = gmm.fit_predict(distance_matrix)
    return cluster_labels

def detect_outliers_elliptic_envelope(features, contamination=0.1):
    scaler = StandardScaler()
    features_std = scaler.fit_transform(features)

    ee = EllipticEnvelope(contamination=contamination)
    ee.fit(features_std)
    
    outlier_pred = ee.predict(features_std)
    
    outliers = np.where(outlier_pred == -1)[0]
    
    decision_scores = ee.decision_function(features_std)
    
    return outliers, decision_scores

def find_largest_object_in_clusters(features_sizes, cluster_labels, class_labels):
    cluster_largest_objects = {}
    areas = np.prod(features_sizes, axis=1)
    unique_clusters = np.unique(cluster_labels)
    for cluster in unique_clusters:
        cluster_indices = np.where(cluster_labels == cluster)[0]
        cluster_areas = areas[cluster_indices]
        
        largest_index_within_cluster = cluster_indices[np.argmax(cluster_areas)]
        largest_class_label = class_labels[largest_index_within_cluster]
        cluster_largest_objects[cluster] = {
            "index": largest_index_within_cluster,
            "area": cluster_areas.max(),
            "label": largest_class_label
        }
    
    return cluster_largest_objects

def build_tree(cluster_largest_objects, cluster_labels, features_sizes, class_labels, label_names):
    cluster_trees = {}
    for cluster, info in cluster_largest_objects.items():
        root_label = label_names[info['label']]
        root_index = info['index']
        
        # Initialize the root node of the tree with the largest object
        tree = {root_index: {"label": root_label, "children": []}}
        cluster_indices = np.where(cluster_labels == cluster)[0]
        
        for idx in cluster_indices:
            if idx == root_index:
                continue  # skip the root node
            
            child_label = label_names[class_labels[idx]]
            
            # Attach all items directly under the root node
            tree[root_index]["children"].append({"index": idx, "label": child_label, "children": []})
        
        cluster_trees[cluster] = tree

    return cluster_trees

def main(scene_id):
    processed_path = '/home/gauravr/Desktop/IFA/code/preprocessed/3d_front_npz/'
    if not os.path.exists(processed_path):
        print(f"Processed path {processed_path} does not exist.")
        sys.exit(1)
    
    all_paths = [os.path.join(processed_path, dir_) for dir_ in os.listdir(processed_path)]
    
    all_scenes = []
    for path_ in all_paths:
        if os.path.isdir(path_):
            all_scenes.extend([os.path.join(path_, x) for x in os.listdir(path_) if os.path.isdir(os.path.join(path_, x))])
    
    scene_path = fetch_scene_id(all_scenes, scene_id)
    if scene_path is None:
        print(f"Scene ID {scene_id} not found.")
        sys.exit(1)

    required_attribs = ['class_labels', 'translations', 'sizes', 'angles']
    attribs = load_attributes_from_npz(scene_path, required_attribs)
    
    for attrib in required_attribs:
        if attrib not in attribs:
            print(f"Attribute '{attrib}' not found in the NPZ file.")
            sys.exit(1)
    
    features_translations, features_sizes = extract_features_from_bboxes(attribs)

    total_objects = attribs['class_labels'].shape[0]
    print(f"Total number of objects in the scene: {total_objects}")

    distance_matrix = dist_matrix(features_translations, features_sizes)

    n_components = min(2, total_objects)
    cluster_labels = cluster_scene_objects_gmm(distance_matrix, n_components=n_components)

    class_labels = attribs['class_labels'].argmax(-1)

    print("\nClass labels for all objects in the scene:")
    for i, label_idx in enumerate(class_labels):
        print(f"Object {i}: {class_labels_bedroom[label_idx]} (Class Index: {label_idx})")

    print(f"\nClustering results using GMM:")
    cluster_dict = {}

    for idx, label in enumerate(cluster_labels):
        if label not in cluster_dict:
            cluster_dict[label] = []
        cluster_dict[label].append(idx)

    for label, indices in cluster_dict.items():
        print(f"\nCluster {label} contains the following objects:")
        for idx in indices:
            print(f"  Object {idx}: {class_labels_bedroom[class_labels[idx]]}")

    print("\nDetecting outliers using Elliptic Envelope...")
    features = np.hstack((features_translations, features_sizes))
    outliers, decision_scores = detect_outliers_elliptic_envelope(features)
    
    print(f"\nOutliers detected at indices: {outliers}")
    print(f"Elliptic Envelope decision scores: {decision_scores}")

    cluster_largest_objects = find_largest_object_in_clusters(features_sizes, cluster_labels, class_labels)

    print("\nLargest objects in each cluster by size:")
    for cluster, info in cluster_largest_objects.items():
        label_name = class_labels_bedroom[info['label']]
        print(f"Cluster {cluster}: Largest Object Index {info['index']} with Area {info['area']} and Label '{label_name}'")

    cluster_trees = build_tree(cluster_largest_objects, cluster_labels, features_sizes, class_labels, class_labels_bedroom)

    for cluster, tree in cluster_trees.items():
        print(f"\nCluster {cluster} Tree Structure:")
        for node, data in tree.items():
            print(f"Root: Object {node} ({data['label']})")
            for child in data["children"]:
                print(f"  └─ Child Object {child['index']} ({child['label']})")

if __name__ == "__main__":
    main(scene_id='MasterBedroom-71933')

Total number of objects in the scene: 12

Class labels for all objects in the scene:
Object 0: wardrobe (Class Index: 20)
Object 1: wardrobe (Class Index: 20)
Object 2: single_bed (Class Index: 15)
Object 3: nightstand (Class Index: 12)
Object 4: nightstand (Class Index: 12)
Object 5: cabinet (Class Index: 2)
Object 6: chair (Class Index: 4)
Object 7: chair (Class Index: 4)
Object 8: table (Class Index: 18)
Object 9: pendant_lamp (Class Index: 13)
Object 10: pendant_lamp (Class Index: 13)
Object 11: tv_stand (Class Index: 19)

Clustering results using GMM:

Cluster 1 contains the following objects:
  Object 0: wardrobe
  Object 2: single_bed
  Object 3: nightstand
  Object 4: nightstand
  Object 5: cabinet
  Object 7: chair
  Object 8: table

Cluster 0 contains the following objects:
  Object 1: wardrobe
  Object 6: chair
  Object 9: pendant_lamp
  Object 10: pendant_lamp
  Object 11: tv_stand

Detecting outliers using Elliptic Envelope...

Outliers detected at indices: [ 9 10]
Ellipti

/home/gauravr/anaconda3/envs/physgen/lib/python3.9/site-packages/sklearn/covariance/_robust_covariance.py:187: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-3.184751963521167 > -36.224537516639444). You may want to try with a higher value of support_fraction (current value: 0.750).
  warnings.warn(
/home/gauravr/anaconda3/envs/physgen/lib/python3.9/site-packages/sklearn/covariance/_robust_covariance.py:187: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-3.184751963521167 > -36.224537516639444). You may want to try with a higher value of support_fraction (current value: 0.750).
  warnings.warn(
/home/gauravr/anaconda3/envs/physgen/lib/python3.9/site-packages/sklearn/covariance/_robust_covariance.py:187: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-3.184751963521167 > -36.224537516639444). You may want to try with a higher value of su